In [29]:
#!/usr/bin/env python3
# -*- coding: utf-8 -*-
"""
Created on Tue Nov  1 08:46:06 2022

@author: schakrab
"""

# for this tutorial we will use netCDF4, xarray, numpy, os, ipywidgets, and matplotlib.pyplot
import netCDF4 as nc
import ipywidgets as widgets
import xarray as xr
import numpy as np
import os
import matplotlib.pyplot as plt
import datetime
from ipywidgets import interactive, interact, Layout, VBox, HBox

from IPython.display import display, clear_output

# set standard font size to 20
plt.rcParams.update({'font.size': 20}) 
  






# get a list of the files in the current directory
INIT     = 'HOU'#######THIS LINE is where the function for the Cities would be#####
path = '/Users/schakrab/Desktop/Paper/input_folder/outputs/HOU/pyout/'####################CHANGE###########
#path = '/Users/schakrab/Desktop/Paper/input_folder/outputs/matout/'
files = os.listdir(path)

# loop through these files
for name in files: 
    if name.endswith('.nc'): 
        # save the filenames into variables for sensitivities
        if "_sens_PM"   in name: pm_sens_filename = path + name
        if "_sens_O3"   in name: o3_sens_filename = path + name
        if "_sens_NO"   in name: no_sens_filename = path + name     
        # for health scaling
        if "_health_PM" in name: pm_health_filename = path + name
        if "_health_O3" in name: o3_health_filename = path + name
        if "_health_NO" in name: no_health_filename = path + name  
        # for uncertainty
        if "_health_lb_PM" in name: pm_health_lb_filename = path + name
        if "_health_lb_O3" in name: o3_health_lb_filename = path + name
        if "_health_lb_NO" in name: no_health_lb_filename = path + name          
        if "_health_ub_PM" in name: pm_health_ub_filename = path + name
        if "_health_ub_O3" in name: o3_health_ub_filename = path + name
        if "_health_ub_NO" in name: no_health_ub_filename = path + name    
            
            
path = '/Users/schakrab/Desktop/Paper/input_folder/static/nei/'
files = os.listdir(path)           
for name in files: 
    if name.endswith('.nc'):             
        # for emissions    
        if "_onroad_" in name:
            if "_onroad_catx" in name:
                onrc_emis_filename = path + name
            else:
                onr_emis_filename = path + name
        if "_nonroad_"    in name: nonr_emis_filename = path + name               
        if "_ag"          in name: agr_emis_filename  = path + name            
        if "_ptnonipm"    in name: ind_emis_filename  = path + name      
        if "_egu_"        in name: egu_emis_filename  = path + name  
            




# read masks####################CHANGE###########
MU = np.loadtxt("/Users/schakrab/Desktop/Paper/input_folder/outputs/HOU/pyout/CITY_05x06.csv", comments="#", delimiter=",", unpack=False)
SU = np.loadtxt("/Users/schakrab/Desktop/Paper/input_folder/outputs/HOU/pyout/STATE_05x06.csv", comments="#", delimiter=",", unpack=False)
####################CHANGE###########




# The state mask is equivalent to the city + county + state. Areas that are double counted are set to one.
SU = MU  + SU
SU[SU > 1] = 1
# Create a mask of ones for the whole domain
DU = SU * 0 +1

names = ['Municipal', 'State','North America']
masks = [MU, SU, DU]



# read in emissions data from the onroad, onroad_catx, and nonroad files and combine them
onr = xr.open_dataset(onr_emis_filename) + xr.open_dataset(onrc_emis_filename) + xr.open_dataset(nonr_emis_filename)
# read in emissions data from the agriculture file
agr = xr.open_dataset(agr_emis_filename)
# read in emissions data from the industry file
ind = xr.open_dataset(ind_emis_filename)
# read in emissions data from the egu file
egu = xr.open_dataset(egu_emis_filename)
# read in adjoint sensitivity for PM2.5, O3, and NO2 to precursor species
pm_sens = xr.open_dataset(pm_sens_filename)
#print(pm_sens)
o3_sens = xr.open_dataset(o3_sens_filename)
no_sens = xr.open_dataset(no_sens_filename)
# read in health scaling files for PM2.5, O3, and NO2
pm_health = xr.open_dataset(pm_health_filename)
o3_health = xr.open_dataset(o3_health_filename)
no_health = xr.open_dataset(no_health_filename)
# read in uncertainty files for PM2.5, O3, and NO2
pm_health_lb = xr.open_dataset(pm_health_lb_filename)
o3_health_lb = xr.open_dataset(o3_health_lb_filename)
no_health_lb = xr.open_dataset(no_health_lb_filename)
pm_health_ub = xr.open_dataset(pm_health_ub_filename)
o3_health_ub = xr.open_dataset(o3_health_ub_filename)
no_health_ub = xr.open_dataset(no_health_ub_filename)
# create an array of sectors emissions
sectors = [onr,egu,agr,ind]



# initialize PM2.5 contributions by taking the BC sens array,
# expand to a fourth sectoral dimension and multiply by 0
# using xarray expand_dims not numpy
print(pm_sens_filename) 
pm_cont = pm_sens.BC.expand_dims({'sectors':4}) * 0
pm_cont_lb = pm_sens.BC.expand_dims({'sectors':4}) * 0
pm_cont_ub = pm_sens.BC.expand_dims({'sectors':4}) * 0
#print(np.shape(pm_cont),np.shape(sect[variable]),np.shape(pm_sens[var]),np.shape(pm_health[var]))
# get precursor species variables loop through species
variables = [species for species in pm_sens]
print(variables)
for var in variables:
    # erase "_PM" from variable name for emissions
    variable = var.replace('_PM','') 
    #print(variable)
    # loop through sectors to calculate sectoral contribution
    for i,sect in enumerate(sectors):
     #   xxx=sect[variable]*pm_sens[var]
        pm_cont[i,:,:,:] = pm_cont[i,:,:,:] + np.multiply(np.multiply(sect[variable],pm_sens[var]),pm_health[var])
        pm_cont_lb[i,:,:,:] = pm_cont_lb[i,:,:,:] + np.multiply(np.multiply(sect[variable],pm_sens[var]),pm_health_lb[var])
        pm_cont_ub[i,:,:,:] = pm_cont_ub[i,:,:,:] + np.multiply(np.multiply(sect[variable],pm_sens[var]),pm_health_ub[var])
       
# perform same calculation for O3
o3_cont = pm_sens.BC.expand_dims({'sectors':4}) * 0
o3_cont_lb = pm_sens.BC.expand_dims({'sectors':4}) * 0
o3_cont_ub = pm_sens.BC.expand_dims({'sectors':4}) * 0
variables = [species for species in o3_sens]
for var in variables:
    variable = var.replace('_O3','') 
    for i,sect in enumerate(sectors):
        o3_cont[i,:,:,:] = o3_cont[i,:,:,:] + np.multiply(np.multiply(sect[variable],o3_sens[var]),o3_health[var])         
        o3_cont_lb[i,:,:,:] = o3_cont_lb[i,:,:,:] + np.multiply(np.multiply(sect[variable],o3_sens[var]),o3_health_lb[var])
        o3_cont_ub[i,:,:,:] = o3_cont_ub[i,:,:,:] + np.multiply(np.multiply(sect[variable],o3_sens[var]),o3_health_ub[var])

        
              
# perform same calculation for NO2            
no_cont = pm_sens.BC.expand_dims({'sectors':4}) * 0
no_cont_lb = pm_sens.BC.expand_dims({'sectors':4}) * 0
no_cont_ub = pm_sens.BC.expand_dims({'sectors':4}) * 0
variables = [species for species in no_sens]
for var in variables:
    variable = var.replace('_NO2','') 
    for i,sect in enumerate(sectors):
        no_cont[i,:,:,:] = no_cont[i,:,:,:] + np.multiply(np.multiply(sect[variable],no_sens[var]),no_health[var])         
        no_cont_lb[i,:,:,:] = no_cont_lb[i,:,:,:] + np.multiply(np.multiply(sect[variable],no_sens[var]),no_health_lb[var])
        no_cont_ub[i,:,:,:] = no_cont_ub[i,:,:,:] + np.multiply(np.multiply(sect[variable],no_sens[var]),no_health_ub[var])

#save into nc files 


base_dir = '/Users/schakrab/Desktop/Paper/input_folder/'
#City Initials
INIT     = 'HOU'####################CHANGE###########
#City Name
NAME     = 'Houston'####################CHANGE###########
#State city is in
STATE    = 'Texas'####################CHANGE###########
SEX = 'both'
IIPAR     = 91
JJPAR     = 89
dmax      = 396
import pandas as pd
from netCDF4 import Dataset,num2date,date2num
import datetime
yy = '2010';mm='12';
date_series_min = pd.date_range(yy+'-'+mm, periods=396, freq='D') # This is DatetimeIndex, perhaps ordinary datetime objects are preferred...
datesout = date_series_min.to_pydatetime() 
unout = 'days since 2010-12-01 00:00:00'
#Set-Up
########################################################################################################################################################
#command lines### list of names of the cities and state ### Hardparse
########################################################################################################################################################
#Directories

out_dir  = base_dir+'outputs/'+INIT+'/'
fn =out_dir+INIT+'_PM_NO_O3.nc'
ds = nc.Dataset(fn, 'w', format='NETCDF4')
    
    # % define dimensions
time = ds.createDimension('time', dmax)
lat = ds.createDimension('lat', JJPAR)
lon = ds.createDimension('lon', IIPAR)
sector =ds.createDimension('sector', 4)  


# timevar = ds.createVariable('time','float32',('time'));timevar.setncattr('units',unout);
# lats = ds.createVariable('lat', 'f4', ('lat',))
# lons = ds.createVariable('lon', 'f4', ('lon',))
# Sectorss = ds.createVariable('sector', 'f4', ('Sector',))

#timevar[:]= date2num(datesout,unout);
value = ds.createVariable('pm_cont', 'f4', ( 'sector','time','lat','lon'))
value[:,:,:,:]=pm_cont
value = ds.createVariable('pm_cont_lb', 'f4', ( 'sector','time','lat','lon'))
value[:,:,:,:]=pm_cont_lb
value = ds.createVariable('pm_cont_ub', 'f4', ( 'sector','time','lat','lon'))
value[:,:,:,:]=pm_cont_ub
value = ds.createVariable('o3_cont', 'f4', ( 'sector','time','lat','lon'))
value[:,:,:,:]=o3_cont
value = ds.createVariable('o3_cont_lb', 'f4', ( 'sector','time','lat','lon'))
value[:,:,:,:]=o3_cont_lb
value = ds.createVariable('o3_cont_ub', 'f4', ( 'sector','time','lat','lon'))
value[:,:,:,:]=o3_cont_ub
value = ds.createVariable('no_cont', 'f4', ( 'sector','time','lat','lon'))
value[:,:,:,:]=no_cont
value = ds.createVariable('no_cont_lb', 'f4', ( 'sector','time','lat','lon'))
value[:,:,:,:]=no_cont_lb
value = ds.createVariable('no_cont_ub', 'f4', ( 'sector','time','lat','lon'))
value[:,:,:,:]=no_cont_ub
# lats[:]=latgc
# lons[:]=longc
ds.close()
# pm_cont = np.transpose(Dataset(out_dir +'LOS_PM_NO_O3.nc', mode='r').variables['pm_cont'][:])
# pm_cont_lb = np.transpose(Dataset(out_dir +'LOS_PM_NO_O3.nc', mode='r').variables['pm_cont_lb'][:])
# pm_cont_ub = np.transpose(Dataset(out_dir +'LOS_PM_NO_O3.nc', mode='r').variables['pm_cont_ub'][:])









        
# create slider widgets  
a=widgets.IntSlider(min=-100, max=100, step=5, value=0,description='Transport:',layout=widgets.Layout(width='100%')) 
clear_output()
b=widgets.IntSlider(min=-100, max=100, step=5, value=0,description='Energy:',layout=widgets.Layout(width='100%'))
c=widgets.IntSlider(min=-100, max=100, step=5, value=0,description='Agriculture:',layout=widgets.Layout(width='100%'))
d=widgets.IntSlider(min=-100, max=100, step=5, value=0,description='Industry:',layout=widgets.Layout(width='100%'))
# create species selector 
species = widgets.Select(options=['PM25','O3','NO2'],description='Species:',disabled=False)
# specify handle colors of sliders
a.style.handle_color = 'lightgreen' 
b.style.handle_color = 'lightblue' 
c.style.handle_color = 'red' 
d.style.handle_color = 'black'  





def f(a, b, c, d, species): 
    # create a list of scale values from widgets
    scales = [a,b,c,d]
    # species specific information
    if species == 'PM25':
        J = pm_cont
        J_lb = pm_cont_lb
        J_ub = pm_cont_ub        
        ymax = 3000
        lab = ' Deaths'
        longlab = 'Premature Deaths'
        levels = np.linspace(-30,30,21)
    if species == 'O3':
        J = o3_cont
        J_lb = o3_cont_lb
        J_ub = o3_cont_ub          
        ymax = 1000
        lab = ' Deaths'        
        longlab = 'Premature Deaths'        
        levels = np.linspace(-10,10,21)
    if species == 'NO2':
        J = no_cont
        J_lb = no_cont_lb
        J_ub = no_cont_ub          
        ymax = 35000
        lab = ' Cases'        
        longlab = 'New Pediatric Asthma Cases'
        levels = np.linspace(-150,150,21)
    # intialize and calculate contribution for pollutant health impacts
    dJ = J[0,:,:,:].sum(axis=0)*0                
    dJ_lb = J_lb[0,:,:,:].sum(axis=0)*0 
    dJ_ub = J_ub[0,:,:,:].sum(axis=0)*0     
    for i, scale in enumerate(scales):
        dJ = dJ + J[i,:,:,:].sum(axis=0)*scale
        dJ_lb = dJ_lb + J_lb[i,:,:,:].sum(axis=0)*scale
        dJ_ub = dJ_ub + J_ub[i,:,:,:].sum(axis=0)*scale        
    # intialize and calculate contributions from specific regions
    values = [0,0,0]
    values_lb = [0,0,0]
    values_ub = [0,0,0]    
    for i,mask in enumerate(masks):
        values[i] = np.multiply(dJ,mask).sum()/100
        values_lb[i] = np.multiply(dJ_lb,mask).sum()/100
        values_ub[i] = np.multiply(dJ_ub,mask).sum()/100        
    # create bar plot
#    plt.close()
#    fig, ax = plt.subplots(figsize=(10,20))  
    fig = plt.figure(figsize=(10,20))
    ax = fig.add_subplot(2, 1, 1)
    bar1 = plt.bar(names,values,color=['cyan', 'red', 'green', 'blue', 'cyan'])    
    error = [[0 for x in range(3)] for y in range(2)]
    error[0][0]=values[0]-values_lb[0]
    error[1][0]=values_ub[0]-values[0]
    error[0][1]=values[1]-values_lb[1]
    error[1][1]=values_ub[1]-values[1]       
    error[0][2]=values[2]-values_lb[2]
    error[1][2]=values_ub[2]-values[2]       
    plt.errorbar(names,values,error,ls='none',capsize=10,ecolor=[0,0,0])
    plt.title('Pollution Health Impacts in City')
    plt.ylabel('Change in Annual '+longlab)
    plt.ylim(-ymax, ymax)    
    ax.axhline(0, color='grey', linewidth=0.8)    
    # Add counts above the two bar graphs
    for c,rect in enumerate(bar1):
        height = rect.get_height()
        if height < 0:
            plt.text(rect.get_x() + rect.get_width() / 2.0, height, f'{height:.0f}'+lab, ha='center', va='top')            
        else:
            plt.text(rect.get_x() + rect.get_width() / 2.0, height, f'{height:.0f}'+lab, ha='center', va='bottom')        
    

    ax = fig.add_subplot(2, 1, 2)
    plt.contourf(dJ/100,levels)
    plt.set_cmap('coolwarm')
    plt.colorbar()
    plt.show()
    
    
    
    
# create interactive plot    
#plt.close('all')
interactive_plot = interactive(f,a=a,b=b,c=c,d=d,species=species)
clear_output()
# specify size of plot by taking the output
output = interactive_plot.children[-1]
output.layout.height = '1200px'
output.layout.width = '800px'
# display output in "HBox"
HBox([interactive_plot])    